# TLSI DL3_V2
transfer learning on aggregated, labeled dataset

In [284]:

import os
import sys
import importlib
import pickle
import keras
import pdb
import pandas as pd
import numpy as np
from datetime import date, datetime
from sklearn.model_selection import KFold

In [285]:
print(keras.__version__)

2.12.0


In [286]:
from google.colab import drive
drive.mount('/content/gdrive')  #can be ignore when run on server

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [287]:
execution_path = "../../"
import os
os.path.abspath(os.getcwd())

'/content'

In [288]:
#GLOBALS AND FILEPATHS#
source_quantile = "0.2"

source_file_path = "/content/gdrive/MyDrive/ResearchProject/data/labeled_normal_data/new_TLSI_input/all_pids_q=0.2.npy"


display(source_file_path)

'/content/gdrive/MyDrive/ResearchProject/data/labeled_normal_data/new_TLSI_input/all_pids_q=0.2.npy'

In [289]:
sys.path.insert(0,'/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/')

In [290]:
from TL_Packages import *

Architecture_GROUP = "TL3_2f_28_32f_20f_20_1"

def runtime():
    print("Runtime:", datetime.now().hour, "o' clock and", datetime.now().minute, "minutes")

def run_tlsi(file):
    """
        run transfer learning algorithm on sample imbalanced data set
    """
    pass

In [291]:
data_ndarr = np.load(source_file_path, allow_pickle=True)
print("data_ndarr type :", type(data_ndarr))

datadict = data_ndarr.item()
print("datadict type :", type(datadict))

data_ndarr type : <class 'numpy.ndarray'>
datadict type : <class 'dict'>


In [292]:
p_ids = datadict.keys()
smaple_size_dict={}
print("patients: n =", len(p_ids), end='\n\n')

for p_id in p_ids:
    df = datadict[p_id]
    # drop column from input data file
    #df.drop(['id','date','am_pef_org','BMI','sex'], axis=1, inplace=True) #ALL DROPS SHOULD OCCUR HERE
    df.drop(['id','date','am_pef_org','BMI','sex', 'age'], axis=1, inplace=True) #,'id','date','am_pef_org','BMI',  'sex', 'age'
    df = np.asarray(df).astype(np.float32)
    print(p_id, "shape:", df.shape)#, end="\t\t")
    smaple_size_dict[p_id]=df.shape[0]

datadict[p_id].head(4)

patients: n = 26

SB-001 shape: (188, 28)
SB-003 shape: (180, 28)
SB-008 shape: (178, 28)
SB-011 shape: (210, 28)
SB-012 shape: (194, 28)
SB-022 shape: (204, 28)
SB-023 shape: (194, 28)
SB-025 shape: (117, 28)
SB-028 shape: (92, 28)
SB-033 shape: (53, 28)
SB-037 shape: (210, 28)
SB-043 shape: (191, 28)
SB-056 shape: (191, 28)
SB-059 shape: (203, 28)
SB-060 shape: (88, 28)
SB-070 shape: (202, 28)
SB-071 shape: (124, 28)
SB-073 shape: (161, 28)
SB-078 shape: (153, 28)
SB-079 shape: (142, 28)
SB-080 shape: (181, 28)
SB-081 shape: (34, 28)
SB-082 shape: (137, 28)
SB-083 shape: (206, 28)
SB-089 shape: (49, 28)
SB-112 shape: (180, 28)


,income,road_dist,cooking,y_am_pef,tempin,humidin,pm25in,co2in,tempdiffin,humidiffin,...,windsd,humid,varp,dewpt,airp,seap,solrhr,solramnt,grdt,class
1,0.857,0.0,0.167,0.558,0.216,0.513,0.358,0.678,0.562,0.364,...,0.107,0.668,0.174,0.471,0.822,0.819,0.165,0.003,0.206,0.0
2,0.857,0.0,0.167,0.006,0.299,0.521,0.411,0.700,0.436,0.672,...,0.171,0.388,0.190,0.491,0.676,0.670,0.603,0.165,0.383,0.0
3,0.857,0.0,0.167,0.078,0.420,0.713,0.372,0.918,0.469,0.821,...,0.515,0.359,0.197,0.481,0.519,0.518,0.485,0.104,0.346,1.0
4,0.857,0.0,0.167,0.428,0.438,0.468,0.224,0.762,0.560,0.490,...,0.338,0.527,0.175,0.467,0.596,0.596,0.491,0.141,0.339,0.0


In [293]:
smaple_size_dict

{'SB-001': 188,
 'SB-003': 180,
 'SB-008': 178,
 'SB-011': 210,
 'SB-012': 194,
 'SB-022': 204,
 'SB-023': 194,
 'SB-025': 117,
 'SB-028': 92,
 'SB-033': 53,
 'SB-037': 210,
 'SB-043': 191,
 'SB-056': 191,
 'SB-059': 203,
 'SB-060': 88,
 'SB-070': 202,
 'SB-071': 124,
 'SB-073': 161,
 'SB-078': 153,
 'SB-079': 142,
 'SB-080': 181,
 'SB-081': 34,
 'SB-082': 137,
 'SB-083': 206,
 'SB-089': 49,
 'SB-112': 180}

#### dropped columns

`date` was dropped above, because it would throw `ValueError: could not convert string to float: '2017-11-01'`
`id` and `am_pef_org` were dropped because neither are explanatory variables.

(Note: `am_pef_org` is the UNSHIFTED, NON-NORMALIZED value for a person's morning exposure. It is relatively raw, and should almost exclusively be used for classification purposes)

In [294]:


target_grp = ["SB-028", "SB-011", "SB-112", "SB-001", "SB-012", "SB-073", "SB-078", "SB-080", "SB-083", "SB-043", "SB-037", "SB-022", "SB-025", "SB-060", "SB-023", "SB-071", "SB-033", "SB-079", "SB-056", "SB-059", "SB-003", "SB-089", "SB-082", "SB-008", "SB-070"]

# SEVERITY_GROUP = 'All-DL3-005-20-1f'


### severity group A
#target_grp = ["SB-028", "SB-011", "SB-112", "SB-001", "SB-012", "SB-073", "SB-078", "SB-080", "SB-083"]
#SEVERITY_GROUP = 'A9'

#target_grp = ["SB-043", "SB-037", "SB-022", "SB-025", "SB-060", "SB-023", "SB-071"]
#SEVERITY_GROUP = 'B7'

#target_grp = ["SB-033", "SB-079", "SB-056", "SB-059", "SB-003", "SB-089", "SB-082", "SB-008", "SB-070"]
#SEVERITY_GROUP = 'C9'

# severity better one

#target_grp = ["SB-028", "SB-011", "SB-112", "SB-001", "SB-073", "SB-043", "SB-071", "SB-012", "SB-078"]
#SEVERITY_GROUP = 'AA9'

#target_grp = ["SB-079", "SB-080", "SB-083", "SB-037", "SB-022", "SB-059", "SB-003", "SB-089", "SB-025"]
#SEVERITY_GROUP = 'BB8-1'

#target_grp = ["SB-060", "SB-023", "SB-033", "SB-056", "SB-082", "SB-008", "SB-070"]
#SEVERITY_GROUP = 'CC7'

# road

#target_grp = ["SB-112", "SB-060", "SB-011", "SB-037", "SB-056", "SB-008", "SB-025", "SB-023", "SB-001", "SB-033"]
#SEVERITY_GROUP = 'D10'

#target_grp = ["SB-112", "SB-060", "SB-011", "SB-037", "SB-056", "SB-008", "SB-025", "SB-023", "SB-001", "SB-033", "SB-078"]
#SEVERITY_GROUP = 'D11'

#target_grp = ["SB-078", "SB-080", "SB-073", "SB-079", "SB-028", "SB-012", "SB-083"]
#SEVERITY_GROUP = 'E7'

#target_grp = ["SB-043", "SB-022", "SB-071", "SB-003", "SB-059", "SB-070", "SB-082", "SB-089"]
#SEVERITY_GROUP = 'F8'


# combination better

#target_grp = ["SB-112", "SB-011", "SB-001", "SB-043", "SB-012", "SB-028", "SB-078", "SB-073"]
#SEVERITY_GROUP = 'I8-DL3-005-20'

#target_grp = ["SB-071", "SB-037", "SB-025", "SB-080", "SB-003", "SB-056", "SB-083", "SB-059"]
#SEVERITY_GROUP = 'G8-DL3-005-20'


#target_grp = ["SB-089", "SB-023", "SB-022", "SB-008", "SB-079", "SB-060", "SB-033", "SB-070", "SB-082"]
#SEVERITY_GROUP = 'H9-DL3-005-20'


# cooking
#target_grp = ["SB-028", "SB-043", "SB-112", "SB-025", "SB-080", "SB-012", "SB-003"]
#SEVERITY_GROUP = "S7-DL2-005"


#target_grp = ["SB-011", "SB-037", "SB-056", "SB-070", "SB-023", "SB-083", "SB-059", "SB-082"]
#SEVERITY_GROUP = "T8-DL-005"

#target_grp = ["SB-001", "SB-089", "SB-008", "SB-033", "SB-022", "SB-071", "SB-073", "SB-060", "SB-078", "SB-079"]
#SEVERITY_GROUP = "R10-a-no"


# income
#target_grp = ["SB-001", "SB-056", "SB-078", "SB-023", "SB-003", "SB-071", "SB-073"]
#SEVERITY_GROUP = "V7"

#target_grp = ["SB-012", "SB-011", "SB-008", "SB-060", "SB-043", "SB-080", "SB-037", "SB-083", "SB-033", "SB-022"]
#SEVERITY_GROUP = "W10"

#target_grp = ["SB-112", "SB-025", "SB-089", "SB-059", "SB-070", "SB-082", "SB-079", "SB-028"]
#SEVERITY_GROUP = "Z8"


In [295]:
EVALUATION_METRICS = ["Weighted Accuracy", "Sensitivity/Recall", "Specificity", "Precision_class0", "Precision_class1", "Precision_avg", "F1_class0", "F1_class1", "F1_avg", "auc_roc_score", "False_Discovery_Rate", "False_Negative_Rate", "False_Omission_Rate", "False_Positive_Rate", "Jaccard"]

In [296]:
# from datetime import datetime
# path_date = datetime.now().strftime("%B_%d_%Y")

RESULT_DIR = Path(f"/content/gdrive/MyDrive/ResearchProject/TL_results")

display(RESULT_DIR)

PosixPath('/content/gdrive/MyDrive/ResearchProject/TL_results')

In [297]:
%run  '/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py'

<Figure size 640x480 with 0 Axes>

In [298]:
%run  '/content/gdrive/MyDrive/ResearchProject/BalancingAlgorithms/Balance.py'

In [299]:
minority_variable=0

# TL_Base_Model

In [300]:
def TL_Base_Model(POPULATION_EPOCH_SIZE,PATIENT_EPOCH_SIZE, p_ids):
  #Target MOdel dataframe
    df_evaluation_results = pd.DataFrame(columns=EVALUATION_METRICS)
  #Population model results
    pop_results=pd.DataFrame(columns=EVALUATION_METRICS)
    minor_var=minority_variable
    for target in tqdm(p_ids):
        if target not in target_grp:
            continue
        runtime()
        print("~~~~~~~~~~~", POPULATION_EPOCH_SIZE, "~~~~~~~~~~~~~~~~~~~", PATIENT_EPOCH_SIZE, "~~~~~~~~~~~~~~~~~~~", target, "~~~~~~~~~~~~~~~~~~~")

        patient_dir = RESULT_DIR / "diagrams" / target

        # create directory for this patient
        if not (patient_dir.exists() and patient_dir.is_dir()):
            os.makedirs(patient_dir)
        CLASS_VAR='class'
        sample_size=smaple_size_dict[target]
        # load population data
        X_train, X_test, Y_train, Y_test = split_data(target=target, data=datadict,  class_var=CLASS_VAR, minority_var=minor_var,test_frac=0.05,
                                                      transfer=True, balance=True)
        print(f"X_train = {len(X_train)}, y_train = {len(Y_train)}")
        assert len(X_train) == len(Y_train), "X_train and Y_train have different lengths."
        #num_folds = min(5, len(X_train))
        #print(num_folds)
        # k-fold cross-validation for population model
        kf_pop = KFold(n_splits=3, shuffle=True)
        population_models = []
        fold_number=0
        for train_idx, val_idx in kf_pop.split(X_train):
             # Check if the indices are within bounds
            assert train_idx.max() < len(X_train), f"train_idx exceeds bounds of X_train (size: {len(X_train)})"
            assert val_idx.max() < len(X_train), f"val_idx exceeds bounds of X_train (size: {len(X_train)})"


            X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
            Y_train_fold, Y_val_fold = Y_train[train_idx], Y_train[val_idx]
            print(f"Population Fold {fold_number}: Training data size = {len(X_train_fold)}, Validation data size = {len(X_val_fold)}")
            net = network_tl(X_train_fold, Y_train_fold, n_hidden=32, learning_rate=0.001, device='cpu')
            net.set_Y_tind(Y_val_fold)
            ##creating population model
            net.make_pop()
            net.train(n_iters=POPULATION_EPOCH_SIZE)
            #val_predictions = net.nn_pop.predict(X_val_fold)
            #val_loss = net.calculate_loss(val_predictions)

            #population_models.append((net, val_loss))


        # Select the best population model based on validation performance (lowest mean validation loss)
        #best_pop_model, _ = min(population_models, key=lambda x: np.mean(x[1]))
        # Save the best population model
        ## save the fold's population model
            VERSION_TAG = f'TLSI_{Architecture_GROUP}_{POPULATION_EPOCH_SIZE}_{PATIENT_EPOCH_SIZE}_POP_FOLD_{fold_number}'
            #best_pop_model_name = f"{target}-best_{VERSION_TAG}.hdf5"
            #best_pop_model_path = patient_dir / best_pop_model_name
            #best_pop_model.save_model(best_pop_model_path)
            #save model
            modelname = f"{target}-{VERSION_TAG}.hdf5"
            model_path=patient_dir / modelname
            net.nn_pop.save(model_path) #nn_pop
          #save training curve
            training_curve_file = patient_dir / f'{VERSION_TAG}_training_curve_{target}'
            net.save_training_curve(training_curve_file)
          ## calculate results for population model
            results, c_matrix = net.pop_test(X_val_fold, Y_val_fold)
           # load target patient data

            dataset_list = split_data(target=target, data=datadict,  class_var=CLASS_VAR, minority_var=minor_var, #explanatory_var
                                balance=True)
           # setup for k-fold cross-validation
            kfold_evaluation_results_list = []
            kfold_confusion_matrix_list = []

            for i in range(2):
            # load the target data in the current round
              X_train_ind, X_test_ind, Y_train_ind, Y_test_ind = dataset_list[i]

            # Make and train a copy of the population model with
            # all but the last layer frozen.
              net.make_ind()
              net.retrain(X_train_ind, Y_train_ind, n_iters=PATIENT_EPOCH_SIZE)
              print("done RE-training", i, "of 3", end=". ")

            # test
              VERSION_TAG = f'TLSI_{Architecture_GROUP}_{POPULATION_EPOCH_SIZE}_{PATIENT_EPOCH_SIZE}_POP_FOLD{fold_number}_IND_FOLD{i}'
            #save model
              modelname = f"{target}-{VERSION_TAG}.hdf5"
              model_path=patient_dir / modelname
              net.nn_ind.save(model_path) #nn_pop

              #save retraining curve
              training_curve_file = patient_dir / f'{VERSION_TAG}_retraining_curve_{target}'
              net.save_training_curve(training_curve_file)

            # print("Calling net.test Now")
            # print("X_test",X_test)
            # print("Y_test",Y_test)
            ## results for individual model for this particular fold
              results, c_matrix = net.test(X_test_ind, Y_test_ind)
            # print("results",results)
            ## adding up the results(evaluation metrics) of all the folds of individual model
              kfold_evaluation_results_list += [results]
            ## adding up the results of confusion matrix for all the folds of individual model
              kfold_confusion_matrix_list += [c_matrix]

              print("done testing", i, "of 2.")

            ## version tag for target model confusion matrix
            TARGET_VERSION_TAG = f'TLSI_{Architecture_GROUP}_{POPULATION_EPOCH_SIZE}_{PATIENT_EPOCH_SIZE}_target_POP_FOLD_{fold_number}'
        ##save file tag for target model of particular pop fold
            target_confusion_matrixes_file = (f"{str(patient_dir)}/{TARGET_VERSION_TAG}_confusion_matrix_{target}")
        ###save file tag for Population model of particular pop fold
            #population_confusion_matrixes_file = (f"{str(patient_dir)}/{VERSION_TAG}_confusion_matrix_{target}")
        ## dataframe for the average of all the indivudial models results
            avg_results = pd.DataFrame(kfold_evaluation_results_list, columns=EVALUATION_METRICS).mean().values
            avg_results = avg_results.reshape(1, 15)
            ##dataframe for population model results for a particular pop model fold'
            results=results.reshape(1,15)
            pop_results=pd.DataFrame(results, columns=EVALUATION_METRICS)
            pop_results.index = [f"{target}_{sample_size}_size_{POPULATION_EPOCH_SIZE}_{PATIENT_EPOCH_SIZE}_POP_FOLD_{fold_number}"]
            ## dataframe for individual model average results for a particular population fold
            df = pd.DataFrame(avg_results, columns=EVALUATION_METRICS)
            df.index = [f"{target}_{sample_size}_size_{POPULATION_EPOCH_SIZE}_{PATIENT_EPOCH_SIZE}_Target_With_POP_FOLD_{fold_number}"]
            ## concatinating the pop results dataframe and avg target results dataframe for a particular population fold
            df_evaluation_results = pd.concat([df_evaluation_results,pop_results,df], axis = 0)
            ##saving target model confusion matrix
            net.save_confusion_matrix(kfold_confusion_matrix_list, target_confusion_matrixes_file)
            ##saving population model confusion matrix
            #net.save_confusion_matrix(c_matrix, population_confusion_matrixes_file)
            fold_number+=1

    # output_path = os.path.join(RESULT_DIR, f'TLSI_{Architecture_GROUP}_evaluation_metrics_{sorted(set(p_ids).difference(target_grp))[0]}.csv')
    output_path = os.path.join(RESULT_DIR, f'TLSI_{Architecture_GROUP}_evaluation_metrics.csv')

    if os.path.isfile(output_path):
        df_evaluation_results.to_csv(output_path,
                                        mode='a',
                                        header=False
                                    )
    else:
                df_evaluation_results.to_csv(output_path,
                                        mode='a'
                                    )

    print("evaluation results saved for", VERSION_TAG)

In [301]:
print()

In [302]:
#POPULATION_EPOCH_SIZE = [*range(500, 1600, 500)]
#PATIENT_EPOCH_SIZE = [*range(500, 1600, 500)]
POPULATION_EPOCH_SIZE = [*range(100, 700, 200)]
PATIENT_EPOCH_SIZE = [*range(100, 700, 200)]
# POPULATION_EPOCH_SIZE = [*range(1, 6, 2)]
# PATIENT_EPOCH_SIZE = [*range(1, 6, 2)]

print("POPULATION_EPOCH_SIZE:",POPULATION_EPOCH_SIZE, "Len:", len(POPULATION_EPOCH_SIZE))
print("PATIENT_EPOCH_SIZE:",PATIENT_EPOCH_SIZE, "Len:", len(PATIENT_EPOCH_SIZE))
print("target_grp:",target_grp, "Len:", len(target_grp))
print("Total records in each file:",len(POPULATION_EPOCH_SIZE) * len(PATIENT_EPOCH_SIZE) * len(target_grp))

POPULATION_EPOCH_SIZE: [100, 300, 500] Len: 3
PATIENT_EPOCH_SIZE: [100, 300, 500] Len: 3
target_grp: ['SB-028', 'SB-011', 'SB-112', 'SB-001', 'SB-012', 'SB-073', 'SB-078', 'SB-080', 'SB-083', 'SB-043', 'SB-037', 'SB-022', 'SB-025', 'SB-060', 'SB-023', 'SB-071', 'SB-033', 'SB-079', 'SB-056', 'SB-059', 'SB-003', 'SB-089', 'SB-082', 'SB-008', 'SB-070'] Len: 25
Total records in each file: 225


# Execution

In [303]:
TL_Base_Model(POPULATION_EPOCH_SIZE[0],PATIENT_EPOCH_SIZE[0], p_ids)

  0%|          | 0/26 [00:00<?, ?it/s]

Runtime: 4 o' clock and 42 minutes
~~~~~~~~~~~ 100 ~~~~~~~~~~~~~~~~~~~ 100 ~~~~~~~~~~~~~~~~~~~ SB-001 ~~~~~~~~~~~~~~~~~~~
24/24 [==============================] - 0s 1ms/step
~~~~~~~ value_counts: ~~~~~~~
1.0    2932
0.0    1496
Name: class, dtype: int64
X_train = 4428, y_train = 4428
Population Fold 0: Training data size = 2952, Validation data size = 1476
47/47 [==============================] - 0s 1ms/step
cm [[281 202]
 [ 64 929]]
cm.ravel() [281 202  64 929]
1/1 [==============================] - 0s 86ms/step
~~~~~~~ value_counts: ~~~~~~~
1.0    74
0.0    40
Name: class, dtype: int64
1/1 [==============================] - 0s 78ms/step
~~~~~~~ value_counts: ~~~~~~~
1.0    77
0.0    34
Name: class, dtype: int64
3/3 [==============================] - 0s 3ms/step
cm [[ 0 17]
 [ 1 76]]
cm.ravel() [ 0 17  1 76]
done testing 0 of 2.
3/3 [==============================] - 0s 3ms/step
cm [[ 2 18]
 [ 5 69]]
cm.ravel() [ 2 18  5 69]
done testing 1 of 2.


/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:585: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_xticklabels([''] + ['0', '1'], fontsize='xx-large')
/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:586: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_yticklabels([''] + ['0', '1'], fontsize='xx-large')


Population Fold 1: Training data size = 2952, Validation data size = 1476
47/47 [==============================] - 0s 1ms/step
cm [[315 201]
 [ 53 907]]
cm.ravel() [315 201  53 907]
1/1 [==============================] - 0s 92ms/step
~~~~~~~ value_counts: ~~~~~~~
1.0    73
0.0    42
Name: class, dtype: int64
1/1 [==============================] - 0s 76ms/step
~~~~~~~ value_counts: ~~~~~~~
1.0    78
0.0    32
Name: class, dtype: int64
3/3 [==============================] - 0s 2ms/step
cm [[ 0 16]
 [ 2 76]]
cm.ravel() [ 0 16  2 76]
done testing 0 of 2.
3/3 [==============================] - 0s 3ms/step
cm [[ 0 21]
 [ 0 73]]
cm.ravel() [ 0 21  0 73]
done testing 1 of 2.


/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:585: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_xticklabels([''] + ['0', '1'], fontsize='xx-large')
/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:586: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_yticklabels([''] + ['0', '1'], fontsize='xx-large')


...

/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:585: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_xticklabels([''] + ['0', '1'], fontsize='xx-large')
/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:586: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_yticklabels([''] + ['0', '1'], fontsize='xx-large')


Population Fold 1: Training data size = 3057, Validation data size = 1529
48/48 [==============================] - 0s 1ms/step
cm [[291 233]
 [ 67 938]]
cm.ravel() [291 233  67 938]
1/1 [==============================] - 0s 83ms/step


 35%|███▍      | 9/26 [14:09<26:45, 94.42s/it]

~~~~~~~ value_counts: ~~~~~~~
1.0    22
0.0     8
Name: class, dtype: int64


IndexError: ignored

In [ ]:
TL_Base_Model(POPULATION_EPOCH_SIZE[0],PATIENT_EPOCH_SIZE[1], p_ids)

  0%|          | 0/26 [00:00<?, ?it/s]

Runtime: 4 o' clock and 41 minutes
~~~~~~~~~~~ 100 ~~~~~~~~~~~~~~~~~~~ 300 ~~~~~~~~~~~~~~~~~~~ SB-001 ~~~~~~~~~~~~~~~~~~~
X_train = 5878, y_train = 5878
Population Fold 0: Training data size = 3918, Validation data size = 1960
62/62 [==============================] - 0s 1ms/step
cm [[856 120]
 [342 642]]
cm.ravel() [856 120 342 642]
3/3 [==============================] - 0s 3ms/step
cm [[ 5 18]
 [26 45]]
cm.ravel() [ 5 18 26 45]
done testing 0 of 2.
3/3 [==============================] - 0s 2ms/step
cm [[ 3 11]
 [26 54]]
cm.ravel() [ 3 11 26 54]
done testing 1 of 2.


/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:582: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_xticklabels([''] + ['0', '1'], fontsize='xx-large')
/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:583: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_yticklabels([''] + ['0', '1'], fontsize='xx-large')


Population Fold 1: Training data size = 3919, Validation data size = 1959
62/62 [==============================] - 0s 1ms/step
cm [[824 157]
 [285 693]]
cm.ravel() [824 157 285 693]
3/3 [==============================] - 0s 3ms/step
cm [[ 5  8]
 [25 56]]
cm.ravel() [ 5  8 25 56]
done testing 0 of 2.
3/3 [==============================] - 0s 3ms/step
cm [[10 14]
 [22 48]]
cm.ravel() [10 14 22 48]
done testing 1 of 2.


/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:582: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_xticklabels([''] + ['0', '1'], fontsize='xx-large')
/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:583: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_yticklabels([''] + ['0', '1'], fontsize='xx-large')


...

Population Fold 1: Training data size = 3921, Validation data size = 1961
62/62 [==============================] - 0s 1ms/step
cm [[779 213]
 [191 778]]
cm.ravel() [779 213 191 778]
3/3 [==============================] - 0s 3ms/step
cm [[11  9]
 [20 50]]
cm.ravel() [11  9 20 50]
done testing 0 of 2.
3/3 [==============================] - 0s 3ms/step
cm [[ 8  8]
 [27 47]]
cm.ravel() [ 8  8 27 47]
done testing 1 of 2.


/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:582: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_xticklabels([''] + ['0', '1'], fontsize='xx-large')
/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:583: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_yticklabels([''] + ['0', '1'], fontsize='xx-large')


Population Fold 2: Training data size = 3922, Validation data size = 1960
62/62 [==============================] - 0s 1ms/step
cm [[791 183]
 [206 780]]
cm.ravel() [791 183 206 780]
3/3 [==============================] - 0s 4ms/step
cm [[ 6 13]
 [28 43]]
cm.ravel() [ 6 13 28 43]
done testing 0 of 2.
3/3 [==============================] - 0s 3ms/step
cm [[ 7 10]
 [25 48]]
cm.ravel() [ 7 10 25 48]
done testing 1 of 2.


/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:582: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_xticklabels([''] + ['0', '1'], fontsize='xx-large')
/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:583: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_yticklabels([''] + ['0', '1'], fontsize='xx-large')
100%|██████████| 26/26 [25:00<00:00, 57.72s/it]


evaluation results saved for TLSI_TL3_2f_28_32f_20f_20_1_100_300_POP_FOLD2_IND_FOLD1


In [ ]:
TL_Base_Model(POPULATION_EPOCH_SIZE[0],PATIENT_EPOCH_SIZE[2], p_ids)

  0%|          | 0/26 [00:00<?, ?it/s]

Runtime: 5 o' clock and 12 minutes
~~~~~~~~~~~ 100 ~~~~~~~~~~~~~~~~~~~ 500 ~~~~~~~~~~~~~~~~~~~ SB-001 ~~~~~~~~~~~~~~~~~~~
X_train = 5872, y_train = 5872
Population Fold 0: Training data size = 3914, Validation data size = 1958
62/62 [==============================] - 0s 1ms/step
cm [[822 145]
 [251 740]]
cm.ravel() [822 145 251 740]
3/3 [==============================] - 0s 3ms/step
cm [[10  7]
 [23 54]]
cm.ravel() [10  7 23 54]
done testing 0 of 2.
3/3 [==============================] - 0s 3ms/step
cm [[ 5 15]
 [11 63]]
cm.ravel() [ 5 15 11 63]
done testing 1 of 2.


/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:582: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_xticklabels([''] + ['0', '1'], fontsize='xx-large')
/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:583: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_yticklabels([''] + ['0', '1'], fontsize='xx-large')


Population Fold 1: Training data size = 3915, Validation data size = 1957
62/62 [==============================] - 0s 1ms/step
cm [[786 199]
 [199 773]]
cm.ravel() [786 199 199 773]
3/3 [==============================] - 0s 2ms/step
cm [[ 6 10]
 [19 59]]
cm.ravel() [ 6 10 19 59]
done testing 0 of 2.
3/3 [==============================] - 0s 3ms/step
cm [[12  9]
 [15 58]]
cm.ravel() [12  9 15 58]
done testing 1 of 2.


/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:582: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_xticklabels([''] + ['0', '1'], fontsize='xx-large')
/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:583: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_yticklabels([''] + ['0', '1'], fontsize='xx-large')


...

Population Fold 1: Training data size = 3940, Validation data size = 1970
62/62 [==============================] - 0s 1ms/step
cm [[828 165]
 [215 762]]
cm.ravel() [828 165 215 762]
3/3 [==============================] - 0s 3ms/step
cm [[ 6 15]
 [21 48]]
cm.ravel() [ 6 15 21 48]
done testing 0 of 2.
3/3 [==============================] - 0s 3ms/step
cm [[ 8  7]
 [19 56]]
cm.ravel() [ 8  7 19 56]
done testing 1 of 2.


/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:582: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_xticklabels([''] + ['0', '1'], fontsize='xx-large')
/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:583: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_yticklabels([''] + ['0', '1'], fontsize='xx-large')


Population Fold 2: Training data size = 3940, Validation data size = 1970
62/62 [==============================] - 0s 1ms/step
cm [[797 178]
 [223 772]]
cm.ravel() [797 178 223 772]
3/3 [==============================] - 0s 2ms/step
cm [[ 7  8]
 [21 54]]
cm.ravel() [ 7  8 21 54]
done testing 0 of 2.
3/3 [==============================] - 0s 3ms/step
cm [[12  9]
 [21 48]]
cm.ravel() [12  9 21 48]
done testing 1 of 2.


/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:582: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_xticklabels([''] + ['0', '1'], fontsize='xx-large')
/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:583: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_yticklabels([''] + ['0', '1'], fontsize='xx-large')
100%|██████████| 26/26 [29:10<00:00, 67.34s/it]

evaluation results saved for TLSI_TL3_2f_28_32f_20f_20_1_100_500_POP_FOLD2_IND_FOLD1


In [ ]:
TL_Base_Model(POPULATION_EPOCH_SIZE[1],PATIENT_EPOCH_SIZE[0], p_ids)

  0%|          | 0/26 [00:00<?, ?it/s]

Runtime: 5 o' clock and 48 minutes
~~~~~~~~~~~ 300 ~~~~~~~~~~~~~~~~~~~ 100 ~~~~~~~~~~~~~~~~~~~ SB-001 ~~~~~~~~~~~~~~~~~~~
X_train = 5874, y_train = 5874
Population Fold 0: Training data size = 3916, Validation data size = 1958
62/62 [==============================] - 0s 1ms/step
cm [[741 213]
 [177 827]]
cm.ravel() [741 213 177 827]
3/3 [==============================] - 0s 2ms/step
cm [[ 9  9]
 [28 48]]
cm.ravel() [ 9  9 28 48]
done testing 0 of 2.
3/3 [==============================] - 0s 3ms/step
cm [[ 6 13]
 [28 47]]
cm.ravel() [ 6 13 28 47]
done testing 1 of 2.


/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:582: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_xticklabels([''] + ['0', '1'], fontsize='xx-large')
/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:583: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_yticklabels([''] + ['0', '1'], fontsize='xx-large')


Population Fold 1: Training data size = 3916, Validation data size = 1958
62/62 [==============================] - 0s 1ms/step
cm [[782 190]
 [186 800]]
cm.ravel() [782 190 186 800]
3/3 [==============================] - 0s 3ms/step
cm [[ 5 16]
 [17 56]]
cm.ravel() [ 5 16 17 56]
done testing 0 of 2.
3/3 [==============================] - 0s 3ms/step
cm [[ 4 12]
 [18 60]]
cm.ravel() [ 4 12 18 60]
done testing 1 of 2.


/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:582: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_xticklabels([''] + ['0', '1'], fontsize='xx-large')
/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:583: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_yticklabels([''] + ['0', '1'], fontsize='xx-large')


...

Population Fold 1: Training data size = 3928, Validation data size = 1964
62/62 [==============================] - 0s 1ms/step
cm [[738 220]
 [159 847]]
cm.ravel() [738 220 159 847]
3/3 [==============================] - 0s 2ms/step
cm [[ 4 18]
 [ 6 62]]
cm.ravel() [ 4 18  6 62]
done testing 0 of 2.
3/3 [==============================] - 0s 3ms/step
cm [[ 9  5]
 [13 63]]
cm.ravel() [ 9  5 13 63]
done testing 1 of 2.


/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:582: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_xticklabels([''] + ['0', '1'], fontsize='xx-large')
/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:583: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_yticklabels([''] + ['0', '1'], fontsize='xx-large')


Population Fold 2: Training data size = 3928, Validation data size = 1964
62/62 [==============================] - 0s 2ms/step
cm [[810 183]
 [217 754]]
cm.ravel() [810 183 217 754]
3/3 [==============================] - 0s 3ms/step
cm [[ 8 11]
 [22 49]]
cm.ravel() [ 8 11 22 49]
done testing 0 of 2.
3/3 [==============================] - 0s 3ms/step
cm [[ 8  9]
 [25 48]]
cm.ravel() [ 8  9 25 48]
done testing 1 of 2.


/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:582: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_xticklabels([''] + ['0', '1'], fontsize='xx-large')
/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:583: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_yticklabels([''] + ['0', '1'], fontsize='xx-large')
100%|██████████| 26/26 [44:31<00:00, 102.76s/it]

evaluation results saved for TLSI_TL3_2f_28_32f_20f_20_1_300_100_POP_FOLD2_IND_FOLD1


In [ ]:
TL_Base_Model(POPULATION_EPOCH_SIZE[1],PATIENT_EPOCH_SIZE[1], p_ids)

  0%|          | 0/26 [00:00<?, ?it/s]

Runtime: 21 o' clock and 17 minutes
~~~~~~~~~~~ 300 ~~~~~~~~~~~~~~~~~~~ 300 ~~~~~~~~~~~~~~~~~~~ SB-001 ~~~~~~~~~~~~~~~~~~~
X_train = 5872, y_train = 5872
Population Fold 0: Training data size = 3914, Validation data size = 1958
62/62 [==============================] - 0s 1ms/step
cm [[828 156]
 [240 734]]
cm.ravel() [828 156 240 734]
3/3 [==============================] - 0s 3ms/step
cm [[ 5  8]
 [19 62]]
cm.ravel() [ 5  8 19 62]
done testing 0 of 2.
3/3 [==============================] - 0s 4ms/step
cm [[ 8 16]
 [22 48]]
cm.ravel() [ 8 16 22 48]
done testing 1 of 2.


/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:582: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_xticklabels([''] + ['0', '1'], fontsize='xx-large')
/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:583: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_yticklabels([''] + ['0', '1'], fontsize='xx-large')


Population Fold 1: Training data size = 3915, Validation data size = 1957
62/62 [==============================] - 0s 1ms/step
cm [[818 164]
 [249 726]]
cm.ravel() [818 164 249 726]
3/3 [==============================] - 0s 3ms/step
cm [[ 7 10]
 [24 53]]
cm.ravel() [ 7 10 24 53]
done testing 0 of 2.
3/3 [==============================] - 0s 3ms/step
cm [[ 7 13]
 [25 49]]
cm.ravel() [ 7 13 25 49]
done testing 1 of 2.


/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:582: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_xticklabels([''] + ['0', '1'], fontsize='xx-large')
/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:583: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_yticklabels([''] + ['0', '1'], fontsize='xx-large')


...

Population Fold 2: Training data size = 3975, Validation data size = 1987
63/63 [==============================] - 0s 1ms/step
cm [[806 207]
 [168 806]]
cm.ravel() [806 207 168 806]
2/2 [==============================] - 0s 4ms/step
cm [[ 2  9]
 [ 8 43]]
cm.ravel() [ 2  9  8 43]
done testing 0 of 2.
2/2 [==============================] - 0s 4ms/step
cm [[ 6  2]
 [15 39]]
cm.ravel() [ 6  2 15 39]
done testing 1 of 2.


/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:582: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_xticklabels([''] + ['0', '1'], fontsize='xx-large')
/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:583: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_yticklabels([''] + ['0', '1'], fontsize='xx-large')
 65%|██████▌   | 17/26 [34:44<18:24, 122.77s/it]

Runtime: 21 o' clock and 52 minutes
~~~~~~~~~~~ 300 ~~~~~~~~~~~~~~~~~~~ 300 ~~~~~~~~~~~~~~~~~~~ SB-073 ~~~~~~~~~~~~~~~~~~~
X_train = 5936, y_train = 5936
Population Fold 0: Training data size = 3957, Validation data size = 1979
62/62 [==============================] - 0s 1ms/step
cm [[851 141]
 [245 742]]
cm.ravel() [851 141 245 742]
3/3 [==============================] - 0s 2ms/step
cm [[13  7]
 [15 46]]
cm.ravel() [13  7 15 46]
done testing 0 of 2.
3/3 [==============================] - 0s 3ms/step
cm [[ 9 12]
 [18 41]]
cm.ravel() [ 9 12 18 41]
done testing 1 of 2.


/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:582: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_xticklabels([''] + ['0', '1'], fontsize='xx-large')
/content/gdrive/MyDrive/ResearchProject/classification/TL_Model/TL_Packages.py:583: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_yticklabels([''] + ['0', '1'], fontsize='xx-large')


Population Fold 1: Training data size = 3957, Validation data size = 1979


In [ ]:
TL_Base_Model(POPULATION_EPOCH_SIZE[1],PATIENT_EPOCH_SIZE[2], p_ids)

In [ ]:
TL_Base_Model(POPULATION_EPOCH_SIZE[2],PATIENT_EPOCH_SIZE[0], p_ids)

In [ ]:
TL_Base_Model(POPULATION_EPOCH_SIZE[2],PATIENT_EPOCH_SIZE[1], p_ids)

In [ ]:
TL_Base_Model(POPULATION_EPOCH_SIZE[2],PATIENT_EPOCH_SIZE[2], p_ids)